In [1]:
%reset -f

In [2]:
import pandas as pd
import numpy as np
from datetime import datetime
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
import os

In [ ]:
# Define paths

base = os.path.basename(os.getcwd())
path = os.path.abspath(os.getcwd())